In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import unicodedata
import re
import numpy as np
import acquire as a
import prepare as p
import requests
import nltk.sentiment
import model_functions as m

from env import username, password, host
from wordcloud import WordCloud
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yogibexar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yogibexar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yogibexar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Exercises
#### Do your work for this exercise in a file named model.

1) Take the work we did in the lessons further:

2) What other types of models (i.e. different classifcation algorithms) could you use?
How do the models compare when trained on term frequency data alone, instead of TF-IDF values?

In [2]:
df = a.get_news()

In [3]:
df.head()

,title,content,category
0,Case filed against Gauri Khan in UP's Lucknow ...,An FIR has been filed against Gauri Khan and o...,national
1,Messi buys 35 gold iPhones worth ₹1.7 cr for W...,Argentina footballer Lionel Messi has reported...,national
2,It was mayhem: Mark Waugh on Indore pitch for ...,Former Australia batter Mark Waugh criticised ...,national
3,"Asked him for tips, he said 'At the end of the...",Australia left-arm spinner Matthew Kuhnemann h...,national
4,Mockery of Test cricket: Dilip Vengsarkar on I...,After 14 wickets fell on Day 1 of the India-Au...,national


In [4]:
sports_words = p.basic_clean(' '.join(df.content[df.category == 'sports']))
politics_words = p.basic_clean(' '.join(df.content[df.category == 'politcs']))
technology_words = p.basic_clean(' '.join(df.content[df.category == 'technology']))
science_words = p.basic_clean(' '.join(df.content[df.category == 'science']))
entertainment_words = p.basic_clean(' '.join(df.content[df.category == 'entertainment']))
world_words = p.basic_clean(' '.join(df.content[df.category == 'world']))

In [5]:
df.category.value_counts()

national         25
business         25
sports           25
world            25
politics         25
technology       25
startup          25
entertainment    25
miscellaneous    25
hatke            25
science          25
automobile       25
Name: category, dtype: int64

In [6]:
df[df.category == 'technology']

,title,content,category
125,Case filed against Gauri Khan in UP's Lucknow ...,An FIR has been filed against Gauri Khan and o...,technology
126,Messi buys 35 gold iPhones worth ₹1.7 cr for W...,Argentina footballer Lionel Messi has reported...,technology
127,It was mayhem: Mark Waugh on Indore pitch for ...,Former Australia batter Mark Waugh criticised ...,technology
128,"Asked him for tips, he said 'At the end of the...",Australia left-arm spinner Matthew Kuhnemann h...,technology
129,Mockery of Test cricket: Dilip Vengsarkar on I...,After 14 wickets fell on Day 1 of the India-Au...,technology
130,"Counting of votes for Tripura, Meghalaya & Nag...",The counting of votes for Assembly polls in Tr...,technology
131,India calls for faster extradition of fugitive...,At the the anti-corruption working group meeti...,technology
132,Nithyananda's Kailasa participated as NGO & it...,The UN said rape-accused self-styled godman Ni...,technology
133,Ashram flyover likely to be inaugurated on Mar...,Delhi Chief Minister Arvind Kejriwal on Wednes...,technology
134,"Who is Nishad, Indian-origin ex-FTX engineer w...","Nishad Singh, the former co-lead engineer at c...",technology


In [7]:
df['technology'] = df.category.map({'technology' : 'yes'})

In [8]:
df.technology.fillna('no', inplace= True)

In [9]:
df.head()

,title,content,category,technology
0,Case filed against Gauri Khan in UP's Lucknow ...,An FIR has been filed against Gauri Khan and o...,national,no
1,Messi buys 35 gold iPhones worth ₹1.7 cr for W...,Argentina footballer Lionel Messi has reported...,national,no
2,It was mayhem: Mark Waugh on Indore pitch for ...,Former Australia batter Mark Waugh criticised ...,national,no
3,"Asked him for tips, he said 'At the end of the...",Australia left-arm spinner Matthew Kuhnemann h...,national,no
4,Mockery of Test cricket: Dilip Vengsarkar on I...,After 14 wickets fell on Day 1 of the India-Au...,national,no


In [10]:
train, val, test = m.split_data(df, strat= 'technology')

In [11]:
tech_article = train[train.technology=='yes']
non_tech_articles = train[train.technology == 'no']

In [12]:
bag_of_words = CountVectorizer().fit_transform(train.content)
bag_of_words.todense()

matrix([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])